## Matrix and Vocabulary Construction

# Código feito pelos monitores.

In [13]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import math
__author__ = "Thierry Barros"

indice_invertido = {}
dados_TF = {}



In [14]:
news = pd.read_csv("../data/estadao_noticias_eleicao.csv", encoding="utf-8")

In [15]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

In [16]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

#### Removing punctuation

In [17]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

#### Removing stopwords

In [18]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens stopword_ = stopwords.words('portuguese')])

#### Transforming list of lists into one list

In [19]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [20]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consult Bigram Frequency

In [21]:
consultable_matrix = matrix.tocsr()

In [22]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

# Código do lab 1 para pesquisa OR.

In [23]:
def preencher( tupla ):
    palavra, id_texto = tupla
    if palavra.lower() not in dados_TF[id_texto]:
        dados_TF[id_texto][palavra.lower()] = 0
    dados_TF[id_texto][palavra.lower()] += 1
    try:
        indice_invertido [ palavra.lower() ].add(id_texto)
    except KeyError:
        indice_invertido[ palavra.lower() ] = set([ id_texto ])

In [24]:

def extrair( texto , id_texto ):
    for i in xrange( len(texto) ):
        dados_TF[id_texto[i]]= {}
        for palavra in nltk.word_tokenize( str(texto[i]).decode('utf-8') ):
            preencher((palavra,id_texto[i]))



#Faça uma busca disjuntiva (OR) considerando a nova consulta.

**Função de busca OR Binário.**
Função de busca que retorna os documentos que possuem pelo menos uma das palavras da consulta. Função de labs anteriores é uma função de busca binária.

In [25]:
def procurarOR(lista):
    indices = indice_invertido[lista[0].lower()]
    for palavra in lista:
        indices = set(reduce( lambda x,y : list( x | y ), [ indice_invertido[palavra.lower()] , indices] ))
    return indices

**Função main** que roda o programa e cria o indíce invertido.

In [26]:
if __name__ == "__main__":
    file_csv = pd.read_csv("../data/estadao_noticias_eleicao.csv")
    file_csv = file_csv.replace(np.nan, '', regex=True)
    texto = file_csv.titulo + " " + file_csv.conteudo + " " + file_csv.subTitulo
    extrair(texto,file_csv.idNoticia)

NameError: name 'xrange' is not defined

# Código do Lab2 - Expansão de Consultas.

#Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorne as top-3 palavras em ordem decrescente de frequencia.

A função pega os tokens(todas as palavras do dicionário), faz um set para tirar as repetições, depois tranforma e lista denovo. Então eu crio uma lista(dic), contendo tuplas de (palavra, frequência da palavra com o termo). Após a criação dessa lista, eu ordeno ela pela frequência, e pego as três palavras com maior frequência e retorno elas. 
OBS: Crio essa lista de nome, frequencia, ultilizando a função auxiliar consult_frequency(word, i), criada pelos monitores da turma, então eu não preciso receber na função a matrix de co-ocorrência, porque essa função já usa a matriz.

In [ ]:
def top3_similarity(word):
    words = set(tokens)
    words = list(words)
    dic = []
    for i in words:
        try:
            dic.append((i, consult_frequency(word, i)))
        except KeyError:
            return ["Not Found: palavra '"+word+"' não encontrada"]
    dic.sort(key=lambda tup: tup[1], reverse=True)
    similar_words = []
    for tupla in dic[:3]:
       similar_words.append(tupla[0])
    return similar_words

#Expanda a consulta original com os termos retornados no passo 2 acima.

Essa função recebe o termo a ser procurado e retorna a lista expandida com os top3 palavras mais similares.

In [ ]:
def expand_consult(word):
    words = []
    words.append(word)
    words += top3_similarity(word)
    return words

##Escolha livremente três termos de consulta e responda o seguinte:

**Os três termos escolhidos foram: Poucos, recursos, corinthians.**

#Quais os termos retornados para a expansão de cada consulta?

In [ ]:
word1 = "poucos"
word2 = "recursos"
word3 = "corinthians"

ex_word1 = top3_similarity(word1)

ex_word2 = top3_similarity(word2)

ex_word3 = top3_similarity(word3)


#Consulta **poucos**:

In [ ]:
print("Termos encontrados na expansão da pesquisa de "+term1+":")
for i, word in enumerate(ex_word1):
    print i+1,word

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, pois são termos que em conjunto fazem sentido. Exemplo: poucos dias, poucos meses, poucos metros.

#Consulta **recursos**:

In [ ]:
print("Termos encontrados na expansão da pesquisa de "+term2+":")
for i, word in enumerate(ex_word2):
    print i+1,word

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, mas acho que só duas das palavras fazem sentido, que são públicos e petrobrás. Recursos fundo não parece fazer muito sentido em conjunto, mas pode ser porque a frase não está completa, poderia ter outras palavras após essas que na frase fariam mais sentido. Já, recursos públicos e recursos petrobrás parece fazer sentido, deve possivelmente está relacionado ao interesse dos usuários em saber como estão sendo gastos os recursos do país.

#Consulta **corinthians**:

In [ ]:
print("Termos encontrados na expansão da pesquisa de "+term3+":")
for i, word in enumerate(ex_word3):
    print i+1,word

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, pois Andrés é o presidente do timão. 2010 deve ter sido a pesquisa em alta no ano da captação desses dados, procurando saber como o corinthians foi ou que jogos ele teria no ano de 2010. Paulo pode ser uma palavra ambígua, oque pode trazer problemas com falsos positivos caso seja uma palavra estatísticamente irrelevante, mas ela tanto pode representar o estado de São paulo, ou algum jogador chamado Paulo que fez sucesso no corinthians na época, confesso que não lembro de nenhum jogador com esse nome.

**Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?**

Para responder essa questão vou pegar **5 documentos** da consulta com o **termo original** e da consulta **expandida** e olhar manualmente quais documentos de quais pesquisas são **mais relevantes para a consulta original**.

#Documentos da consulta **poucos**.

In [ ]:
print(procurarOR([word1])[:5])
print(procurarOR(ex_word1)[:5])

#Documentos da consulta **recursos**.

In [ ]:
print(procurarOR([word2])[:5])
print(procurarOR(ex_word2)[:5])

#Documentos da consulta **corinthians**.

In [ ]:
print(procurarOR([word3])[:5])
print(procurarOR(ex_word3)[:5])

#A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

Para responder essa questão vou checar o número de documentos retornados pelas duas consultas.

In [ ]:
print("N° documentos retornado pela consulta "+word1+":"+len(procurarOR([word1]))
print("N° documentos retornado pela consulta expandida "+ex_word1+":"+len(procurarOR(ex_word1)))

In [ ]:
print("N° documentos retornado pela consulta "+word1+":"+len(procurarOR([word2]))
print("N° documentos retornado pela consulta expandida "+ex_word1+":"+len(procurarOR(ex_word2)))

In [ ]:
print("N° documentos retornado pela consulta "+word1+":"+len(procurarOR([word3]))
print("N° documentos retornado pela consulta expandida "+ex_word1+":"+len(procurarOR(ex_word3)))

Podemos ver que os 3 casos o número de documentos retornados pelas consultas expandidas é bem maior que o número retornado pela consulta só com a palavra original.Isso sempre vai acontecer já que a consulta expandida contém o termo da consulta original e é uma busca por OR, então **no mínimo a consulta expandida trará o mesmo número de documentos que a consulta original**, como podemos ver em geral ela trará mais documentos. Com base nisso, em relação ao **recall**(N° de documentos relevantes retornados dentre todos os relevantes), então podemos ver que o **recall da consulta expandida sempre será maior ou igual ao recall da consulta original**. No entando **sobre a precision não podemos afirmar nada com certeza**, pode ser um trade-off ao mesmo tempo que aumentasmo o recall, aumentando o número de documentos retornados, podemos está diminuindo, ou não, o precision da consulta. Não sabemos quantos dos novos documentos retornados são relevantes para a consulta. Como a consulta binária não faz um raking dos documentos, é difícil afirmar se os documentos são relevantes ou não, mas por eurísticas e testes já feitos anteriormente por pesquisadores da área, foi mostrado que na maioria dos casos a busca expandida melhora o retorno da consulta.